In [1]:
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Flatten
from matplotlib import pyplot
from PIL import Image
import os, sys
from sklearn.model_selection import train_test_split
import numpy

In [2]:
# Define the encoder block based on the original paper
def define_encoder_block(layer, filtersNo, batchnorm=True):
    
    # init weights from a Gaussian distribution with mean 0 and standard deviation 0.02
    init = RandomNormal(stddev=0.02)
    
    # in the original paper, all convolution kernels are (4,4), with stride 2. Stride for decoder means downsampling.
    x = Conv2D(filtersNo, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer)
    
    # Conditional batch normalization (important for the first layer)
    if batchnorm:
        x = BatchNormalization()(x, training=True)
        
    # All ReLUs in the encoder are leaky!
    x = LeakyReLU(alpha=0.2)(x)
    
    return x

In [3]:
# Define the decoder block based on the original paper
def decoder_block(layer, skip, filtersNo, dropout=True, batch=True):
    
    # init weights from a Gaussian distribution with mean 0 and standard deviation 0.02
    init = RandomNormal(stddev=0.02)
    
    # in the original paper, all convolution kernels are (4,4), with stride 2. Stride for decoder means upsampling.
    x = Conv2DTranspose(filtersNo, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer)

    # All layers in the original paper have batch normalization, although we set an if statement just to play with the model
    if batch:
        x = BatchNormalization()(x, training=True)
        
    # Some decoder layers don't have dropout
    if dropout:
        x = Dropout(0.5)(x, training=True)
        
    # Merge with skip connection
    x = Concatenate()([x, skip])
    
    # All ReLUs in the decoder are not leaky!
    x = Activation('relu')(x)
    
    return x

In [4]:
# Define the generator based on encoder/decoder
def define_generator():
    
    # init weights from a Gaussian distribution with mean 0 and standard deviation 0.02
    init = RandomNormal(stddev=0.02)
    
    # image input
    inputImage = Input(shape=(128,128,1))
    
    ###### Encoder
    
    # C64, input (128,128,1), output (64,64,64)
    encoderLayer1 = define_encoder_block(inputImage, 64, batchnorm=False)
    
    #C128, input (64,64,64), output (32,32,128)
    encoderLayer2 = define_encoder_block(encoderLayer1, 128)
    
    #C256, input (32,32,128), output (16,16,256)
    encoderLayer3 = define_encoder_block(encoderLayer2, 256)
    
    #C512, input (16,16,256), output (8,8,512)
    encoderLayer4 = define_encoder_block(encoderLayer3, 512)
    
    #C512, input (8,8,512), output (4,4,512)
    encoderLayer5 = define_encoder_block(encoderLayer4, 512)
    
    #C512, input (4,4,512), output (2,2,512)
    encoderLayer6 = define_encoder_block(encoderLayer5, 512)
    
    ###### Bottleneck layer, will have an input of (2,2,512) and an output of (1,1,512)
    bottleneck = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(encoderLayer6)
    bottleneck = Activation('relu')(bottleneck)
    
    ###### Decoder, with skip connection
    
    #CD512
    decoderLayer1 = decoder_block(bottleneck, encoderLayer6, 512)
    
    #CD512
    decoderLayer2 = decoder_block(decoderLayer1, encoderLayer5, 512)
    
    #C512
    decoderLayer3 = decoder_block(decoderLayer2, encoderLayer4, 512, dropout=False)
    
    #C256
    decoderLayer4 = decoder_block(decoderLayer3, encoderLayer3, 256, dropout=False)
    
    #C128
    decoderLayer5 = decoder_block(decoderLayer4, encoderLayer2, 128, dropout=False)
    
    #C64
    decoderLayer6 = decoder_block(decoderLayer5, encoderLayer1, 64, dropout=False)
    
    # Output with tanh function, as mentioned in the original paper. Output will be (128x128x3)
    g = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(decoderLayer6)
    outputImage = Activation('tanh')(g)
    
    # Define model
    model = Model(inputImage, outputImage)
    
    return model

In [5]:
# Define the 70x70 discriminator as in the original paper
def define_discriminator():
    
    # init weights from a Gaussian distribution with mean 0 and standard deviation 0.02
    init = RandomNormal(stddev=0.02)
    
    # source image input
    source = Input(shape=(128,128,1))
    
    # target image input
    target = Input(shape=(128,128,3))
    
    # concatenate images channel-wise
    merged = Concatenate()([source, target])
    
    # C64
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    
    # C128
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    # C256
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    # C512
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    # patch output
    d = Conv2D(1, (8,8), strides=(8,8), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(d)
    patch_out = Flatten()(patch_out)
    
    # define model
    model = Model([source, target], patch_out)
    
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model

In [6]:
# Define Pix2Pix GAN
def pix2pix(generator, discriminator):
    
    # make weights in the discriminator not trainable
    for layer in discriminator.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
            
    # define the source image
    source = Input(shape=(128,128,1))
    
    # connect the source image to the generator input
    genOut = generator(source)
    
    # connect the source input and generator output to the discriminator input
    disOut = discriminator([source, genOut])
    
    # src image as input, generated image and classification output
    model = Model(source, [disOut, genOut])
    
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    
    return model

In [7]:
def generateFromDataset(trainX, trainY, samples):
    
    # Choose random images from both input and output
    no = randint(0, trainX.shape[0], samples)
    gx, gy = trainX[no], trainY[no]
    
    # Set y-labels to 1, as these images are from dataset
    y = ones((samples, 1))
    return [gx, gy], y

In [8]:
def generateFromGenerator(generator, samples):
    # Generate fake instance
    x = generator.predict(samples)
    
    # Labels will be zero because they come from the generator
    y = zeros((len(x), 1))
    
    return x, y

In [18]:
def summarize_performance(step, g_model, trainX, trainY, n_samples=5):
    # select a sample of input images
    [X_realA, X_realB], _ = generateFromDataset(trainX, trainY, n_samples)
    
    # Generate fake samples
    X_fakeB, _ = generateFromGenerator(g_model, X_realA)
    
    # Plot black and white images
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(X_realA[i])
        
    # Plot generated images
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(X_fakeB[i])
        
    # Plot expected output images
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
        pyplot.axis('off')
        pyplot.imshow(X_realB[i])
        
    # Save plot to file
    filename1 = 'images/plot_%06d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    
    # Save generator model
    filename2 = 'models/model_%06d.h5' % (step+1)
    g_model.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))

In [19]:
def train(discriminator, generator, gan, epochs=100000, samplesPerEpoch=250):
    
    # Load the data
    bwData = load("Flickr8kblackandwhite1dim.npy")
    colorData = load("flickr8k_shuffled.npy")
    y = numpy.ones((8091,1))
    
    BWSplit = numpy.array_split(bwData, 2)
    colorSplit = numpy.array_split(colorData, 2)
    glossMin = 999
    # manually enumerate epochs
    for i in range(epochs):
        
        # Generate real samples
        [realX, realY], realLabel = generateFromDataset(BWSplit[0], colorSplit[0], samplesPerEpoch)
        
        # Generate fake samples
        fakeY, fakeLabel = generateFromGenerator(generator, realX)
        
        # Update discriminator on real samples
        realLoss = discriminator.train_on_batch([realX, realY], realLabel)
        
        # Update discriminator on fake samples
        fakeLoss = discriminator.train_on_batch([realX, fakeY], fakeLabel)
        
        # Update generator
        generatorLoss, _, _ = gan.train_on_batch(realX, [realLabel, realY])
        
        # summarize performance
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, realLoss, fakeLoss, generatorLoss))
        if glossMin > generatorLoss:
            summarize_performance(i, generator, BWSplit[0], colorSplit[0])
            glossMin = generatorLoss

In [20]:
d = define_discriminator()
g = define_generator()
p2p = pix2pix(g,d)

In [ ]:
train(d,g,p2p)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


>1, d1[0.001] d2[2.141] g[14.335]
>Saved: images/plot_000001.png and models/model_000001.h5


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


>2, d1[0.327] d2[0.233] g[13.066]
>Saved: images/plot_000002.png and models/model_000002.h5


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


>6988, d1[0.001] d2[0.009] g[2.701]
>Saved: images/plot_006988.png and models/model_006988.h5
>6989, d1[0.001] d2[0.009] g[3.888]
>6990, d1[0.001] d2[0.013] g[2.861]
>6991, d1[0.001] d2[0.011] g[2.862]
>6992, d1[0.001] d2[0.013] g[2.879]
>6993, d1[0.001] d2[0.026] g[2.722]
>6994, d1[0.001] d2[0.025] g[2.855]
>6995, d1[0.002] d2[0.007] g[3.258]
>6996, d1[0.002] d2[0.005] g[3.710]
>6997, d1[0.002] d2[0.007] g[3.182]
>6998, d1[0.001] d2[0.006] g[2.968]
>6999, d1[0.001] d2[0.004] g[3.167]
>7000, d1[0.001] d2[0.008] g[2.841]
>7001, d1[0.001] d2[0.005] g[2.939]
>7002, d1[0.001] d2[0.008] g[2.792]
>7003, d1[0.001] d2[0.005] g[2.888]
>7004, d1[0.001] d2[0.005] g[3.259]
>7005, d1[0.001] d2[0.004] g[2.925]
>7006, d1[0.001] d2[0.009] g[3.713]
>7007, d1[0.001] d2[0.005] g[3.095]
>7008, d1[0.001] d2[0.005] g[3.878]
>7009, d1[0.001] d2[0.009] g[3.082]
>7010, d1[0.001] d2[0.013] g[4.978]
>7011, d1[0.001] d2[0.005] g[2.845]
>7012, d1[0.001] d2[0.007] g[3.105]
>7013, d1[0.001] d2[0.008] g[2.981]
>7014,

>7214, d1[0.000] d2[0.002] g[3.504]
>7215, d1[0.000] d2[0.001] g[3.774]
>7216, d1[0.000] d2[0.003] g[4.326]
>7217, d1[0.000] d2[0.001] g[3.386]
>7218, d1[0.000] d2[0.001] g[4.156]
>7219, d1[0.000] d2[0.002] g[4.294]
>7220, d1[0.000] d2[0.001] g[3.675]
>7221, d1[0.000] d2[0.002] g[3.302]
>7222, d1[0.000] d2[0.001] g[3.722]
>7223, d1[0.000] d2[0.001] g[4.657]
>7224, d1[0.000] d2[0.001] g[5.635]
>7225, d1[0.000] d2[0.001] g[5.265]
>7226, d1[0.000] d2[0.001] g[6.224]
>7227, d1[0.000] d2[0.001] g[4.038]
>7228, d1[0.000] d2[0.005] g[3.771]
>7229, d1[0.000] d2[0.006] g[3.722]
>7230, d1[0.000] d2[0.002] g[3.513]
>7231, d1[0.000] d2[0.001] g[3.549]
>7232, d1[0.000] d2[0.001] g[3.607]
>7233, d1[0.000] d2[0.001] g[4.093]
>7234, d1[0.000] d2[0.002] g[4.258]
>7235, d1[0.000] d2[0.001] g[4.155]
>7236, d1[0.000] d2[0.001] g[4.264]
>7237, d1[0.000] d2[0.001] g[3.807]
>7238, d1[0.000] d2[0.002] g[3.531]
>7239, d1[0.000] d2[0.001] g[4.583]
>7240, d1[0.000] d2[0.001] g[4.883]
>7241, d1[0.000] d2[0.001] g

>7442, d1[0.000] d2[0.001] g[5.316]
>7443, d1[0.000] d2[0.001] g[3.871]
>7444, d1[0.000] d2[0.001] g[4.488]
>7445, d1[0.000] d2[0.001] g[4.187]
>7446, d1[0.000] d2[0.001] g[5.508]
>7447, d1[0.000] d2[0.003] g[7.079]
>7448, d1[0.000] d2[0.001] g[5.800]
>7449, d1[0.000] d2[0.002] g[5.189]
>7450, d1[0.000] d2[0.001] g[4.881]
>7451, d1[0.000] d2[0.002] g[5.094]
>7452, d1[0.000] d2[0.001] g[4.749]
>7453, d1[0.000] d2[0.002] g[4.039]
>7454, d1[0.000] d2[0.001] g[5.984]
>7455, d1[0.000] d2[0.001] g[4.673]
>7456, d1[0.000] d2[0.001] g[6.404]
>7457, d1[0.000] d2[0.002] g[4.964]
>7458, d1[0.000] d2[0.005] g[6.175]
>7459, d1[0.000] d2[0.001] g[4.877]
>7460, d1[0.000] d2[0.002] g[5.917]
>7461, d1[0.000] d2[0.001] g[5.572]
>7462, d1[0.000] d2[0.002] g[5.704]
>7463, d1[0.000] d2[0.001] g[5.146]
>7464, d1[0.000] d2[0.001] g[4.378]
>7465, d1[0.000] d2[0.001] g[5.548]
>7466, d1[0.000] d2[0.001] g[4.344]
>7467, d1[0.000] d2[0.001] g[6.007]
>7468, d1[0.000] d2[0.001] g[4.364]
>7469, d1[0.000] d2[0.001] g

>7670, d1[0.000] d2[0.001] g[5.255]
>7671, d1[0.000] d2[0.004] g[7.454]
>7672, d1[0.000] d2[0.001] g[5.589]
>7673, d1[0.000] d2[0.001] g[5.328]
>7674, d1[0.000] d2[0.001] g[4.720]
>7675, d1[0.000] d2[0.000] g[7.160]
>7676, d1[0.000] d2[0.000] g[7.829]
>7677, d1[0.000] d2[0.001] g[5.999]
>7678, d1[0.000] d2[0.001] g[6.412]
>7679, d1[0.000] d2[0.005] g[5.929]
>7680, d1[0.000] d2[0.001] g[6.841]
>7681, d1[0.000] d2[0.001] g[6.954]
>7682, d1[0.000] d2[0.000] g[6.079]
>7683, d1[0.000] d2[0.000] g[7.923]
>7684, d1[0.000] d2[0.001] g[5.075]
>7685, d1[0.000] d2[0.000] g[5.683]
>7686, d1[0.000] d2[0.001] g[5.125]
>7687, d1[0.000] d2[0.001] g[7.244]
>7688, d1[0.000] d2[0.001] g[5.926]
>7689, d1[0.000] d2[0.001] g[9.261]
>7690, d1[0.000] d2[0.001] g[9.104]
>7691, d1[0.000] d2[0.001] g[5.259]
>7692, d1[0.000] d2[0.001] g[5.092]
>7693, d1[0.000] d2[0.001] g[6.000]
>7694, d1[0.000] d2[0.001] g[6.381]
>7695, d1[0.000] d2[0.001] g[5.413]
>7696, d1[0.000] d2[0.000] g[5.376]
>7697, d1[0.000] d2[0.000] g